## Import libraries

In [1]:
import torch
from torch.nn import Sequential, ReLU, Module, Dropout, Sigmoid, Linear, BatchNorm2d
from torchvision.datasets import MNIST
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
# necessary parameters
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
batch_size = 512
epochs = 20
learning_rate = 1e-4

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

## Load data

In [4]:
seed = 42   # for reproducibility
torch.manual_seed(seed)     # set seed for torch
torch.backends.cudnn.benchmark = False 
torch.backends.cudnn.deterministic = True 
batch_size = 512
epochs = 20 
learning_rate = 1e-4

### Creating train, val & test loaders

In [10]:
svhn = datasets.SVHN(root='E:/torchvision/datasets', download=False, transform=transforms.ToTensor())

train_data, test_data = train_test_split(svhn, train_size=0.8, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.125, random_state=42)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [14]:
# Checking the shape of the input 

print("Train data")
print(len(train_data))
for batch in train_loader:
    images, labels = batch
    print(images.shape)
    print(labels.shape)
    break

print("\nValidation data")
print(len(val_data))
for batch in val_loader:
    images, labels = batch
    print(images.shape)
    print(labels.shape)
    break

print("\nTest data")
print(len(test_data))
for batch in test_loader:
    images, labels = batch
    print(images.shape)
    print(labels.shape)
    break

Train data
51279
torch.Size([512, 3, 32, 32])
torch.Size([512])

Validation data
7326
torch.Size([512, 3, 32, 32])
torch.Size([512])

Test data
14652
torch.Size([512, 3, 32, 32])
torch.Size([512])


## Creating model architecture

In [27]:
class UnderCompleteAutoencoder(nn.Module):
    
    def __init__(self):
        super(UnderCompleteAutoencoder, self).__init__()

        self.input_dim = 3*32*32
        self.encoder = nn.Sequential(
            nn.Linear(self.input_dim, int(self.input_dim*0.5)),
            nn.ReLU(),
            nn.Linear(int(self.input_dim*0.5), int(self.input_dim*0.25)),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(int(self.input_dim*0.25), int(self.input_dim*0.5)),
            nn.ReLU(),
            nn.Linear(int(self.input_dim*0.5), self.input_dim),
            nn.Sigmoid() 
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [28]:
model = UnderCompleteAutoencoder().to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

## Training undercomplete model

In [32]:
# training step
train_loss = []

for epoch in range(epochs):
    running_loss = 0
    for images, _ in train_loader:

        images = images.view(512, -1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        train_loss = criterion(outputs, images)
        train_loss.backward()
        optimizer.step()
        running_loss += train_loss.item()

    loss = running_loss/len(train_loader)

    print("Epoch : ",epoch+1,"/",epochs,"recon loss = ",loss)